In [ ]:
from openai import OpenAI
import pandas as pd

#init client
client=OpenAI(api_key=your_api_key)
data={
    'review':[
        "The movie was breathtaking and emotional!",
        "This film was a complete waste of time.",
        "Decent acting but the plot was predictable.",
        "I absolutely loved every second of it."

    ]
}

df=pd.DataFrame(data)
df

,review
0,The movie was breathtaking and emotional!
1,This film was a complete waste of time.
2,Decent acting but the plot was predictable.
3,I absolutely loved every second of it.


Zero-Shot Prompring

In [ ]:
for r in df['review']:
  response=client.chat.completions.create(
      model='gpt-3.5-turbo',
      messages=[{"role": "user",
                   "content": f"Classify the sentiment of this review as Positive or Negative:\n\n{r}"}])
  print(r,response.choices[0].message.content.strip())

The movie was breathtaking and emotional! Positive
This film was a complete waste of time. Negative
Decent acting but the plot was predictable. Negative
I absolutely loved every second of it. Positive


One shot prompting

In [ ]:
example="Example:'I hated the movie.'--> Negative"
for r in df['review']:
  response=client.chat.completions.create(
      model='gpt-3.5-turbo',
      messages=[{"role":"user",'content':f"{example}\n Now classify this review:\n{r}"}]  )
  print(r,response.choices[0].message.content.strip())

The movie was breathtaking and emotional! Example: 'The movie was breathtaking and emotional.'--> Positive
This film was a complete waste of time. Negative
Decent acting but the plot was predictable. Negative
I absolutely loved every second of it. Positive


Role Prompting

In [ ]:
for r in df["review"]:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a world-class movie critic who must judge sentiment."},
            {"role": "user", "content": f"Review: {r}\nGive sentiment (Positive/Negative)."}
        ]
    )
    print(r, "→", response.choices[0].message.content.strip())


The movie was breathtaking and emotional! → Sentiment: Positive
This film was a complete waste of time. → Sentiment: Negative
Decent acting but the plot was predictable. → Negative
I absolutely loved every second of it. → Sentiment: Positive.


Chain of Thought prompting

In [ ]:
for r in df["review"]:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user",
                   "content": f"Classify the sentiment. Think step by step and explain reasoning, then give final answer:\n\n{r}"}]
    )
    print(r, "→", response.choices[0].message.content.strip())
    print("\n")
    print("----"*10)


The movie was breathtaking and emotional! → Step 1: Identify key words and phrases
- "breathtaking"
- "emotional"

Step 2: Determine the overall tone conveyed by the words
- The words "breathtaking" and "emotional" suggest positive feelings and a strong emotional impact.

Step 3: Consider the context
- The context is a movie review, indicating that the speaker is expressing their feelings about a movie.

Final answer: Positive sentiment.


----------------------------------------
This film was a complete waste of time. → Step 1: Identify key words and phrases
- "complete waste of time"

Step 2: Determine sentiment
- The use of the phrase "complete waste of time" indicates a negative sentiment.

Final answer: Negative sentiment


----------------------------------------
Decent acting but the plot was predictable. → 1. "Decent acting" - Positive sentiment as the acting was considered good or satisfactory.
2. "But the plot was predictable" - Negative sentiment as the plot was not engaging

Prompting Chaining:Use multi-step prompts (output from one becomes input to another).

In [ ]:
for r in df["review"]:
    # Step 1: Summarize
    summary = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Summarize this review in one line:\n{r}"}]
    ).choices[0].message.content.strip()

    # Step 2: Classify
    sentiment = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": f"Classify this summary as Positive or Negative:\n{summary}"}]
    ).choices[0].message.content.strip()

    print(r, "→", summary, "→", sentiment)


The movie was breathtaking and emotional! → A powerful and emotionally moving film that captivates the audience. → Positive
This film was a complete waste of time. → The film was deemed as a waste of time by the reviewer. → Negative
Decent acting but the plot was predictable. → The film had good acting but lacked a surprising plot. → Negative
I absolutely loved every second of it. → A glowing and enthusiastic endorsement of the experience. → Positive


Negative Example Prompting

In [ ]:
bad_example = "Incorrect: 'I hated the movie.' → Positive"

for r in df["review"]:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user",
                   "content": f"{bad_example}\nThis is wrong. Instead, classify correctly:\n{r}"}]
    )
    print(r, "→", response.choices[0].message.content.strip())


The movie was breathtaking and emotional! → Correct: Positive
This film was a complete waste of time. → Negative
Decent acting but the plot was predictable. → Neutral
I absolutely loved every second of it. → Correct: 'I absolutely loved every second of it.' → Positive


Advance prompt Engineering technique

Chain of Thought:Tell the model: “Think step by step”. Forces reasoning before conclusion.

In [ ]:
question="If i have 3 apples and eat one i have how many apples left?"
response=client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{
        "role": "user",
        "content":f'Answer the following . Think sptep by step ,then give final answer:\n{question}'
    }]
)
print(response.choices[0].message.content)

Step 1: Start with 3 apples. 
Step 2: Eat one apple. 
Step 3: Count how many apples are left. 

Final answer: 2 apples left.


Self-Consistency(COT++)


Don’t trust the first chain. Sample multiple reasoning paths -->pick majority.

In [ ]:
import  collections
answers=[]
for _  in range(5):
  response=client.chat.completions.create(
      model='gpt-3.5-turbo',
      temperature=1.2,
      messages=[{
          "role":"user",
          "content":"Think step by step :If a train leaves at 3 pm and takes 2 hours, what time will it arrive ?"
      }])
  answers.append(response.choices[0].message.content)
final=collections.Counter(answers).most_common(1)[0][0]
print("ALL outputs :",answers)
print("final ",final)

ALL outputs : ['Step 1: The train leaves at 3 pm.\n\nStep 2: The train takes 2 hours to reach its destination.\n\nStep 3: Add 2 hours to the departure time of 3 pm. \n\nStep 4: 3 pm + 2 hours = 5 pm \n\nStep 5: The train will arrive at 5 pm.', 'Step 1: The train leaves at 3 pm and takes 2 hours\nStep 2: To find the arrival time, add the 2 hours of travel time to the departure time of 3 pm\nStep 3: 3 pm + 2 hours = 5 pm\nStep 4: Therefore, the train will arrive at 5 pm.', 'Step 1: The train leaves at 3 pm.\n\nStep 2: The train takes 2 hours to reach its destination.\n\nStep 3: Add 2 hours to the departure time. \n\nStep 4: 3pm + 2 hours = 5pm\n\nStep 5: The train will arrive at its destination at 5 pm.', 'If the train leaves at 3 pm and takes 2 hours, we can add those 2 hours to the departure time of 3 pm:\n\n3 pm + 2 hours = 5 pm\n\nTherefore, the train will arrive at its destination at 5 pm.', '1. The train leaves at 3 pm.\n2. It takes 2 hours to reach its destination.\n3. Add 2 hours

Tree of thoughts(TOT)

nstead of one chain, the model explores multiple reasoning branches at each step → prunes the best.
Good for puzzles, planning, math.

In [ ]:
question = "You have to plan dinner with 3 ingredients. Suggest creative options."
def generate_branches(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1.0,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip().split("\n")

# Root prompt
branches = generate_branches(f"Think of 3 different ideas. Question: {question}")

# Expand each branch
tree = {}
for b in branches:
    tree[b] = generate_branches(f"Refine this idea into 3 variants: {b}")
print(tree)


{'1. Grilled Peach and Prosciutto Salad with Balsamic Glaze:': ['1. Grilled Peach and Prosciutto Salad with a Drizzle of Balsamic Reduction', '2. Balsamic Glazed Grilled Peach and Prosciutto Salad', '3. Summer Salad: Grilled Peach, Prosciutto, and Balsamic Glaze'], '   - Ingredients: Fresh peaches, prosciutto, balsamic glaze': ['1. Grilled Peach and Prosciutto Skewers drizzled with Balsamic Glaze', '2. Peach and Prosciutto Crostini topped with a Balsamic Glaze Reduction', '3. Arugula Salad with Fresh Peaches, Crispy Prosciutto, and Balsamic Glaze Drizzle'], '   - Instructions: Slice the peaches and grill them until they have grill marks. Tear the prosciutto into pieces and toss with the grilled peaches. Drizzle with balsamic glaze and serve as a light and flavorful salad.': ['1. Grilled Peach and Prosciutto Salad: Grill the peaches until caramelized, then combine with torn prosciutto and drizzle with balsamic glaze for a delicious and light salad option.', '', '2. Balsamic Peach and Pr

Embeddings

In [ ]:
pip install openai numpy scikit-learn nltk

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import nltk
nltk.download('punkt')

#init OpenAI client
client=OpenAI(api_key=your_api_key)
#sample dataset
data={
    "review":["The movie was breathtaking and emotional!",
        "This film was a complete waste of time.",
        "Decent acting but the plot was predictable.",
        "I absolutely loved every second of it."]
}
df=pd.DataFrame(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#1.Chatacter-level Embeddings

In [ ]:
def char_embedding(text):
  alphabet="abcdefghijklmnopqrstuvwxyz"
  vec=np.zeros(len(alphabet))
  for c in text.lower():
    if c in alphabet:
      vec[alphabet.index(c)]+=1
  return vec/(np.linalg.norm(vec)+1e-8) #np.linalg.norm(vec) = Euclidean length of the vector.
df["char_vec"]=df['review'].apply(char_embedding)
df['char_vec']

,char_vec
0,"[0.497518594609945, 0.099503718921989, 0.0, 0...."
1,"[0.3216337601065108, 0.0, 0.10721125336883694,..."
2,"[0.2860387765009504, 0.19069251766730028, 0.28..."
3,"[0.1111111109876543, 0.1111111109876543, 0.111..."


#2. Word-Level embedding

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
word_vecs=vectorizer.fit_transform(df['review']).toarray()
df['word_vec']=list(word_vecs)
df['word_vec']

,word_vec
0,"[0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, ..."
2,"[0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, ..."


Sentence-Level Embeddings (OpenAI API)

In [ ]:
sentence_vecs = []
for r in df["review"]:
    emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=r
    ).data[0].embedding
    sentence_vecs.append(emb)
df["sentence_vec"] = sentence_vecs
df["sentence_vec"]

,sentence_vec
0,"[-0.031178174540400505, 0.011328460648655891, ..."
1,"[-0.02348867431282997, 0.0749022513628006, -0...."
2,"[-0.007894045673310757, 0.030286453664302826, ..."
3,"[-0.024015650153160095, 0.016489243134856224, ..."


Compare similarities

In [ ]:
def show_similarities(vectors, label):
    sims = cosine_similarity(vectors)
    print(f"\n🔹 {label} similarity matrix:")
    print(np.round(sims, 2))


In [ ]:

# Character-level similarity
show_similarities(np.vstack(df["char_vec"]), "Character-level")

# Word-level similarity
show_similarities(np.vstack(df["word_vec"]), "Word-level")

# Sentence-level similarity
show_similarities(np.vstack(df["sentence_vec"]), "Sentence-level")


🔹 Character-level similarity matrix:
[[1.   0.82 0.82 0.72]
 [0.82 1.   0.78 0.71]
 [0.82 0.78 1.   0.73]
 [0.72 0.71 0.73 1.  ]]

🔹 Word-level similarity matrix:
[[1.   0.15 0.31 0.  ]
 [0.15 1.   0.14 0.15]
 [0.31 0.14 1.   0.  ]
 [0.   0.15 0.   1.  ]]

🔹 Sentence-level similarity matrix:
[[1.   0.32 0.3  0.41]
 [0.32 1.   0.38 0.36]
 [0.3  0.38 1.   0.22]
 [0.41 0.36 0.22 1.  ]]


Byte Pair Encoding (BPE)

In [ ]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers

texts = ["The movie was fantastic!", "I hated the slow pacing."]

# Train BPE tokenizer
tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=50, special_tokens=["[UNK]", "[PAD]"])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.train_from_iterator(texts, trainer)

for t in texts:
    print(f"{t} {tokenizer.encode(t).tokens}")


The movie was fantastic! ['The', 'movie', 'was', 'fantas', 'tic', '!']
I hated the slow pacing. ['I', 'hated', 'the', 'slow', 'pacing', '.']


Word2Vec (classic)

In [ ]:

import numpy as np
import re
from collections import Counter
from openai import OpenAI


# Tiny dataset
texts = [
    "The movie was fantastic and inspiring.",
    "I hated the slow pacing of the film.",
    "The plot was predictable but the acting was good."
]

# ----------------------------------------
# 1️⃣ Byte Pair Encoding (toy version)
# ----------------------------------------
def byte_pair_tokenize(text, merges=5):
    tokens = list(text.lower().replace(" ", "_"))
    vocab = Counter(tokens)
    for _ in range(merges):
        pairs = Counter(zip(tokens, tokens[1:]))
        if not pairs: break
        most_common = max(pairs, key=pairs.get)
        new_token = "".join(most_common)
        i = 0
        while i < len(tokens)-1:
            if (tokens[i], tokens[i+1]) == most_common:
                tokens[i:i+2] = [new_token]
            i += 1
    return tokens

print("\n🔹 BPE Example:")
for t in texts:
    print(t, "→", byte_pair_tokenize(t))

# ----------------------------------------
# 2️⃣ Word2Vec / GloVe (toy: co-occurrence)
# ----------------------------------------
def co_occurrence_matrix(corpus, window=2):
    vocab = set(word for text in corpus for word in text.split())
    vocab = list(vocab)
    mat = {w: Counter() for w in vocab}
    for text in corpus:
        words = text.split()
        for i, word in enumerate(words):
            context = words[max(i-window,0):i] + words[i+1:i+window+1]
            mat[word].update(context)
    return mat

co_matrix = co_occurrence_matrix(texts)
print("\n🔹 Word2Vec/GloVe style co-occurrence:")
for w, ctx in list(co_matrix.items())[:3]:
    print(w, "→", dict(ctx))

# ----------------------------------------
# 3️⃣ FastText (toy: subword units)
# ----------------------------------------
def fasttext_embedding(word, n=3):
    ngrams = [word[i:i+n] for i in range(len(word)-n+1)]
    return ngrams

print("\n🔹 FastText subwords for 'fantastic':", fasttext_embedding("fantastic"))

# ----------------------------------------
# 4️⃣ Sentence Embeddings (OpenAI)
# ----------------------------------------
embeddings = []
for t in texts:
    emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=t
    ).data[0].embedding
    embeddings.append(emb)

print("\n🔹 Sentence Embedding shape:", len(embeddings[0]))

# ----------------------------------------
# 5️⃣ Graph Embeddings (DeepWalk style, toy)
# ----------------------------------------
graph = {
    "A": ["B","C"],
    "B": ["A","C"],
    "C": ["A","B","D"],
    "D": ["C"]
}

def random_walk(graph, start, length=4):
    walk = [start]
    for _ in range(length-1):
        cur = walk[-1]
        nxt = np.random.choice(graph[cur])
        walk.append(nxt)
    return walk

print("\n🔹 DeepWalk toy walk from A:", random_walk(graph, "A"))



🔹 BPE Example:
The movie was fantastic and inspiring. → ['th', 'e_', 'm', 'o', 'v', 'i', 'e_', 'w', 'as', '_', 'f', 'an', 't', 'as', 't', 'i', 'c', '_', 'an', 'd', '_', 'in', 's', 'p', 'i', 'r', 'in', 'g', '.']
I hated the slow pacing of the film. → ['i_', 'h', 'a', 't', 'e', 'd', '_the_', 's', 'l', 'o', 'w', '_', 'p', 'a', 'c', 'i', 'n', 'g', '_', 'o', 'f', '_the_', 'f', 'i', 'l', 'm', '.']
The plot was predictable but the acting was good. → ['the_', 'p', 'l', 'o', 't_', 'wa', 's', '_', 'p', 'r', 'e', 'd', 'i', 'c', 't', 'a', 'b', 'l', 'e_', 'b', 'u', 't_', 'the_', 'a', 'c', 't', 'i', 'n', 'g', '_', 'wa', 's', '_', 'g', 'o', 'o', 'd', '.']

🔹 Word2Vec/GloVe style co-occurrence:
pacing → {'the': 2, 'slow': 1, 'of': 1}
slow → {'hated': 1, 'the': 1, 'pacing': 1, 'of': 1}
good. → {'acting': 1, 'was': 1}

🔹 FastText subwords for 'fantastic': ['fan', 'ant', 'nta', 'tas', 'ast', 'sti', 'tic']

🔹 Sentence Embedding shape: 1536

🔹 DeepWalk toy walk from A: ['A', np.str_('C'), np.str_('B'), np